# Summary

This notebook explores an enhanced implementation of the VAD processing. The problem was that we were triggering audio recording too early due to false positives. This new implementation uses a sliding window approach (taken from the py-webrtcvad examples file) to reduce false positives thereby improving accuracy.

In [ ]:
import collections
from collections.abc import Iterable

import numpy as np
import sounddevice as sd
import webrtcvad
from scipy.signal import resample_poly


class Frame:
    def __init__(self, bytes, timestamp, duration):
        self.bytes = bytes
        self.timestamp = timestamp
        self.duration = duration

    def __repr__(self):
        return f"Frame(timestamp={self.timestamp}, duration={self.duration}, bytes_length={len(self.bytes)})"


def audio_frame_generator(
    *,
    frame_duration_ms: int,
    sample_rate: int,
    device: int,
) -> Iterable[Frame]:
    """
    Generate audio frames from the microphone.
    """
    duration = frame_duration_ms / 1000.0  # E.g., 0.03 seconds
    frame_size = int(sample_rate * duration)  # E.g., 44100 * 0.03 = 1323 samples
    timestamp = 0.0
    with sd.InputStream(
        samplerate=sample_rate,
        channels=1,
        dtype="int16",
        blocksize=frame_size,
        device=device,
    ) as stream:
        while True:
            audio = stream.read(frame_size)[0]
            yield Frame(audio.tobytes(), timestamp, duration)
            timestamp += duration


def resample(
    audio: np.ndarray, original_sample_rate: int, target_sample_rate: int
) -> np.ndarray:
    """
    This is the equivalent of calling:
    resample_poly(audio, target_sample_rate, original_sample_rate)

    But the program will use less compute resources if we reduce the
    ratio 44100:16000 to 441:160 with np.gcd (Greatest Common Divisor)
    which finds the largest integer that evenly divides two numbers.
    """
    gcd = np.gcd(original_sample_rate, target_sample_rate)
    return resample_poly(audio, target_sample_rate // gcd, original_sample_rate // gcd)


def resample_frames(
    frames: Iterable[Frame],
    original_sample_rate: int,
    target_sample_rate: int,
) -> Iterable[Frame]:
    """
    A generator that wraps an audio frame generator and resamples
    the audio frames.
    """
    for frame in frames:
        audio = np.frombuffer(frame.bytes, dtype=np.int16)
        resampled_audio = resample(audio, original_sample_rate, target_sample_rate)
        # We are changing the sample rate, not the bit-depth.
        # Therefore we can keep the same dtype.
        resampled_audio = np.clip(resampled_audio, -32768, 32767).astype(np.int16)
        yield Frame(resampled_audio.tobytes(), frame.timestamp, frame.duration)


def vad_collector(
    *,
    sample_rate: int,
    frame_duration_ms: int,
    padding_duration_ms: int,
    vad: webrtcvad.Vad,
    frames: Iterable[Frame],
) -> Iterable[bytes]:
    """
    Implementation taken from the example in the py-webrtcvad
    https://github.com/wiseman/py-webrtcvad/blob/master/example.py
    """
    num_padding_frames = int(padding_duration_ms / frame_duration_ms)
    # We use a deque for our sliding window/ring buffer.
    ring_buffer = collections.deque(maxlen=num_padding_frames)
    # We have two states: TRIGGERED and NOTTRIGGERED. We start in the
    # NOTTRIGGERED state.
    triggered = False

    voiced_frames = []
    for frame in frames:
        is_speech = vad.is_speech(frame.bytes, sample_rate)

        print("1" if is_speech else "0")
        if not triggered:
            ring_buffer.append((frame, is_speech))
            num_voiced = len([f for f, speech in ring_buffer if speech])
            # If we're NOTTRIGGERED and more than 90% of the frames in
            # the ring buffer are voiced frames, then enter the
            # TRIGGERED state.
            if num_voiced > 0.9 * ring_buffer.maxlen:
                triggered = True
                print("+(%s)" % (ring_buffer[0][0].timestamp,))
                # We want to yield all the audio we see from now until
                # we are NOTTRIGGERED, but we have to start with the
                # audio that's already in the ring buffer.
                for f, s in ring_buffer:
                    voiced_frames.append(f)
                ring_buffer.clear()
        else:
            # We're in the TRIGGERED state, so collect the audio data
            # and add it to the ring buffer.
            voiced_frames.append(frame)
            ring_buffer.append((frame, is_speech))
            num_unvoiced = len([f for f, speech in ring_buffer if not speech])
            # If more than 90% of the frames in the ring buffer are
            # unvoiced, then enter NOTTRIGGERED and yield whatever
            # audio we've collected.
            if num_unvoiced > 0.9 * ring_buffer.maxlen:
                print("-(%s)" % frame)
                triggered = False
                yield b"".join([f.bytes for f in voiced_frames])
                ring_buffer.clear()
                voiced_frames = []
    if triggered:
        print("-(%s)" % frame)
    print("\n")
    # If we have any leftover voiced audio when we run out of input,
    # yield it.
    if voiced_frames:
        yield b"".join([f.bytes for f in voiced_frames])


frame_duration_ms = 30  # Duration of each frame in milliseconds
mic_sample_rate = 44100  # Original sample rate
vad_sample_rate = 32000  # Sample rate for VAD processing
device = 1  # Device index for the microphone (change as needed)

raw_frame_generator = audio_frame_generator(
    frame_duration_ms=frame_duration_ms,
    sample_rate=mic_sample_rate,
    device=device,
)

resampled_frames = resample_frames(
    raw_frame_generator,
    original_sample_rate=mic_sample_rate,
    target_sample_rate=vad_sample_rate,
)

vad = webrtcvad.Vad(3)
voiced_audio_generator = vad_collector(
    sample_rate=vad_sample_rate,
    frame_duration_ms=frame_duration_ms,
    padding_duration_ms=300,
    vad=vad,
    frames=resampled_frames,
)